Import the required libraries

In [ ]:
! pip install transformers openai numpy requests

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 157 kB 61.8 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 3.3 MB 48.0 MB/s 
     |████████████████████████████████| 52 kB 1.8 MB/s 
     |████████████████████████████████| 636 kB 72.4 MB/s 
     |████████████████████████████████| 11.3 MB 71.1 MB/s 
     |████████████████████████████████| 158 kB 55.7 MB/s 
     |████████████████████████████████| 242 kB 62.9 MB/s 
  Created wheel for openai: filename=openai-0.10.5-py3-none-any.whl size=169225 sha256=9634a188f3222d585b1c2285046744838e1e978ef089c55709829cfbdc5500ed
  Stored in directory: /root/.cache/pip/wheels/c8/85/9b/a634387cd7f6e0ec71ffedd5774d1fbc14f60261eb8aeea6a5
Successfully built openai
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: pandas
    Found exist

The below function returns the contextualized word embeddings from gpt2

In [ ]:
from transformers import GPT2Tokenizer
import numpy as np
import json
import os, openai, requests
#OPENAI_API_KEY = os.environ.get("sk-ckCerHbXrCSeaf61c9ELvsy2EcUEz0w2SxBCsIeD")
OPENAI_API_KEY = 'sk-ckCerHbXrCSeaf61c9ELvsy2EcUEz0w2SxBCsIeD'
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
def get_embeddings(tokens):
   tokens = ', '.join(tokens)
   tokens = "[[" +  tokens + "]]"
   headers = {
       "Authorization": f"Bearer {OPENAI_API_KEY}",
       "Content-Type": "application/json",
   }
   data = (
       '{"prompt":' + f"{tokens}" + '}'
   )
   response = requests.post(
       "https://api.openai.com/v1/engines/ada-search-index-v1/embeddings",
       headers=headers,
       data=data.encode("utf-8"),
   )
   content = json.loads(response.text)
   return content   
   #return content["data"][0]["activations"]


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

The encode_sent function returns the word_encoding after pre-processing

In [ ]:
def encode_sent(sent, type_data):
   tokens = tokenizer(sent)['input_ids']

    # if condition to check if it is a query or a document
   if type_data == 'query':
    print('This is a query')
    tokens = tokenizer("[")['input_ids'] + tokens + tokenizer("]")['input_ids'] # query embeddings
   else:
    tokens = tokenizer("{")['input_ids'] + tokens + tokenizer("}")['input_ids'] # doc embeddings
   if len(tokens) > 2048:
      tokens = tokens[-2048:]
   tokens = [str(t) for t in tokens]
   return tokens


More details related to encode or the tokenizer function

In [ ]:
sent = 'Hello, I hope you are fine'
tokens = tokenizer(sent)
tokens

{'input_ids': [15496, 11, 314, 2911, 345, 389, 3734], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

*   attention_mask = corresponds to the number of words
*   input_ids = corresponds to the word ids from the vocabulary
*   For the sentence - 'Hello, I hope you are fine' has 7 encodings as the tokenised form will contain 7 tokens consisting ('**Hello**', '**,**' , '**I**', '**hope**', '**you**', '**are**', '**fine**') 

In [ ]:
# output from the encode_sent function()
sent = 'Hello, I hope you are fine'
results_encode = encode_sent(sent, type_data='query')
results_encode

This is a query


['58', '15496', '11', '314', '2911', '345', '389', '3734', '60']

The sentence which is sent as q query will always be enclosed in [ query ].In the above encode each of the token or word corresponds to a number


*   [     = 58
*   Hello = 15496
*   ,     = 11
*   I     = 314





In [ ]:
# call the function to get the results
def embed(txt, type_data):
   tokens = encode_sent(txt,type_data)
   return get_embeddings(tokens)
   

In [ ]:
# calling the embed function with a sentence as a parameter
txt = 'Hello, I hope you are fine.'
embed(txt)

{'object': 'list', 'data': [{'object': 'embeddings_result', 'activations': [9.0703125, -0.7060547, -4.9648438, 7.421875, 4.4921875, -3.2851562, 1.0351562, 0.0031604767, -0.5673828, 4.9375, 4.1835938, 0.49316406, 7.2070312, 10.0859375, -1.0585938, -4.8320312, -13.5078125, -2.0136719, 7.2539062, -3.8554688, 0.87646484, 0.42333984, -1.3203125, -0.52685547, -3.7910156, 0.6254883, 0.75097656, 4.5351562, 1.9101562, -2.2988281, -15.171875, -4.671875, -0.65625, -7.1054688, 4.3085938, -5.0117188, -3.1855469, 4.2070312, -1.2216797, 1.5976562, -0.0904541, 8.5, -8.8125, 5.3710938, -3.3730469, 9.984375, -6.8203125, -7.453125, 2.765625, -2.7792969, -0.3466797, 13.34375, 1.3193359, 11.3203125, -2.5234375, -3.9277344, 8.5625, -5.7929688, -7.1523438, -8.4375, -1.3408203, -10.421875, 0.30249023, -4.2421875, -2.2480469, 0.31225586, -8.6171875, 8.71875, 1.3652344, -1.6513672, -1.3681641, 10.8515625, -9.640625, -1.0039062, 8.5078125, 2.1757812, 7.125, -1.0341797, 7.328125, 5.3789062, -2.9277344, 2.4023438,

In [ ]:
# to calculate the cosine similarity
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

In [ ]:
cosine_similarity([results["data"][0]["activations"]],[results1["data"][0]["activations"]])

array([[0.89317742]])

# Build a simple semantic search

Upload the file to extract the word embeddings and repalce with the filename - trial2.json

In [ ]:
document_dictionary = {}

# Python program to read
# json file
import json

# Enter the json file name
f = open('trial2.json',encoding="utf8")

# returns JSON object as a dictionary
data = json.load(f)

counter = 0
# Iterating through the json list
for i in data['lexata']:
    st = 'doc_' + str(counter)
    document_dictionary[st] = i['metadata'], i['text']
    counter = counter + 1

# Closing file
f.close()

In [ ]:
 # create an empty list which will store the word embeddings and will be indexed
word_embed_dictionary = {}

counter = 0

for i in document_dictionary.keys():
  st = 'doc_'+ str(counter)
  results = embed(document_dictionary.get(i)[1], type_data='doc')
  final_results = results["data"][0]["activations"]
  word_embed_dictionary[st] = final_results
  counter = counter + 1

In [ ]:
# create an empty list
word_embed_values = []

for i in word_embed_dictionary.keys():
  word_embed_values.append(word_embed_dictionary.get(i))

textfile = open("embedding_values.txt", "w")
for element in word_embed_values:
    textfile.write(str(element) + "\n")
textfile.close()


In [ ]:
for i in word_embed_values:
  print(i)


[0.15649414, 3.8789062, 1.5234375, 13.75, 3.5722656, -3.7578125, 0.41088867, 4.28125, 3.4082031, -1.2402344, -1.6005859, 0.060638428, -0.640625, 6.3828125, -3.6347656, -3.2265625, 8.9765625, 1.1269531, 0.030288696, 3.6992188, 4.828125, -7.3125, 6.0625, 1.0878906, 6.3945312, -0.8745117, 1.5488281, 6.6992188, -1.8291016, -9.8515625, -3.125, -8.6953125, 0.95947266, 7.8203125, 2.3710938, 5.8984375, -1.7460938, 4.65625, 1.3935547, -2.5820312, -3.0585938, -5.9453125, 7.0195312, 1.6855469, -1.9267578, 2.3339844, -9.6796875, 2.203125, -8.46875, 8.25, 3.7050781, 3.0703125, -4.7890625, 2.4609375, 5.7148438, -5.375, 5.4960938, -0.7817383, -5.0273438, 4.328125, -0.17407227, -0.89501953, 1.7519531, 0.21777344, -6.5664062, 0.6015625, -3.8613281, -5.2851562, 7.3515625, 0.19152832, 2.2558594, 3.5332031, -1.0917969, -9.1875, -4.328125, 2.0117188, 1.2539062, -3.6015625, 1.3847656, 0.63623047, -4.890625, -0.9790039, -1.9013672, -6.484375, -9.6640625, 9.2578125, -8.9765625, -4.5976562, 0.9223633, -3.52734

# query the documents 

In [ ]:
txt = 'IPO venture issuer'
results = embed(txt, type_data='query')
response = results["data"][0]["activations"]

This is a query


In [ ]:
def calculate_cosine_similarity(vec1, vec2):
  return cosine_similarity([vec1],[vec2])  

In [ ]:
# create an empty dictionary
semantic_results = {}

for i in range(len(word_embed_values)):
  st = 'doc_' + str(i)
  res = calculate_cosine_similarity(response, word_embed_values[i])
  semantic_results[st] = res

In [ ]:
semantic_results

{'doc_0': array([[0.32838594]]),
 'doc_1': array([[0.40580551]]),
 'doc_10': array([[0.23801798]]),
 'doc_11': array([[0.26697274]]),
 'doc_12': array([[0.24888566]]),
 'doc_13': array([[0.39279149]]),
 'doc_14': array([[0.25862095]]),
 'doc_15': array([[0.28440699]]),
 'doc_16': array([[0.30150576]]),
 'doc_17': array([[0.29983481]]),
 'doc_18': array([[0.30127175]]),
 'doc_19': array([[0.33466277]]),
 'doc_2': array([[0.36241187]]),
 'doc_20': array([[0.3143425]]),
 'doc_21': array([[0.31924675]]),
 'doc_22': array([[0.29978424]]),
 'doc_23': array([[0.29326355]]),
 'doc_24': array([[0.29394689]]),
 'doc_25': array([[0.28182429]]),
 'doc_26': array([[0.30665565]]),
 'doc_27': array([[0.28523389]]),
 'doc_3': array([[0.31509842]]),
 'doc_4': array([[0.31673168]]),
 'doc_5': array([[0.29682449]]),
 'doc_6': array([[0.32205762]]),
 'doc_7': array([[0.30862633]]),
 'doc_8': array([[0.31917253]]),
 'doc_9': array([[0.29118139]])}

In [ ]:
# sorts the document and displays the results
sort_orders = sorted(semantic_results.items(), key=lambda x: x[1], reverse=True)
for i in sort_orders:
  print('doc id : ', i[0])
  print('semantic_score : ', i[1])
  print(document_dictionary.get(i[0]))
  print('=============================')
  

doc id :  doc_1
semantic_score :  [[0.40580551]]
('10597', 'In this Instrument: IPO venture issuer means an issuer that (a) files a long form prospectus, (b) is not a reporting issuer in any jurisdiction immediately before the date of the final long form prospectus, and (c) at the date of the long form prospectus, does not have any of its securities listed or quoted, has not applied to list or quote any of its securities, and does not intend to apply to list or quote any of its securities, on (i) the Toronto Stock Exchange, (i.1) Aequitas NEO Exchange Inc., (ii) a U.S. marketplace, or (iii) a marketplace outside of Canada and the United States of America, other than the Alternative Investment Market of the London Stock Exchange or the PLUS markets operated by PLUS Markets Group plc; IPO venture issuer')
doc id :  doc_13
semantic_score :  [[0.39279149]]
('10951', 'Lexata note: venture issuer is defined in s.1.1 of NI 41-101, s.1.1 ofÂ\xa0 NI 52-107, s. 1.1 of NI 52-109, and s.1.1 of NI 

In [ ]:
semantic_results.items()

dict_items([('doc_0', array([[0.24075519]])), ('doc_1', array([[0.3001128]]))])